# Strip Chart Testing

This notebook is used for testing the strip chart mode on  Auxiliary Telescope.

Craig Lage - 02-Jul-21

In [1]:
import sys
import asyncio
import time
import os
import numpy as np
from astropy.time import Time, TimeDelta
from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.latiss import LATISS

In [2]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [3]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG
# Make matplotlib less chatty
logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [4]:
#Start classes
domain = salobj.Domain()
latiss = LATISS(domain)
await latiss.start_task

atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.03 sec
Read 1 history items for RemoteEvent(ATHeaderService, 0, authList)
Read 100 history items for RemoteEvent(ATHeaderService, 0, heartbeat)
Read 21 history items for RemoteEvent(ATHeaderService, 0, largeFileObjectAvailable)
Read 1 history items for RemoteEvent(ATHeaderService, 0, logLevel)
Read 100 history items for RemoteEvent(ATHeaderService, 0, logMessage)
Read 1 history items for RemoteEvent(ATHeaderService, 0, simulationMode)
Read 1 history items for RemoteEvent(ATHeaderService, 0, softwareVersions)
Read 11 history items for RemoteEvent(ATHeaderService, 0, summaryState)
Read historical data in 0.07 sec
Read 1 history items for RemoteEvent(ATArchiver, 0, authList)
Read 100 history items for RemoteEvent(ATArchiver, 0, heartbeat)
Read 21 history items for RemoteEvent(ATArchiver, 0, imageInOODS)
Read 21 history ite

[None, None, None, None]

In [ ]:
# enable components
# No need - all enabled
#await latiss.enable({"atspectrograph": "current"})

In [5]:
# Take a bias to make sure everything is working
# Succeeded
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
logMessage DDS read queue is filling: 11 of 100 elements


array([2021070200023])

Step one - go in and out of calibration.\
This succeeded - I could see the camera substate change.

In [9]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [10]:
await asyncio.sleep(2.0)

In [11]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [12]:
# Take a bias to make sure everything is working
# Succeeded
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021070200024])

Step 1.5:
In and out of calibration with a clear.\
Succeeded

In [13]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [14]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [15]:
await asyncio.sleep(2.0)

In [16]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [17]:
# Take a bias to make sure everything is working
# Succeeded
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021070200025])

Step 2:
The following commands should take a calibration image\
Try this first without the discardRows()\
First time, endImage() failed. 

In [18]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [19]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [20]:
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST" 

In [21]:
print(keyValueMap)

groupId: 2021-07-02T18:25:42.210,imageType: ENGTEST


In [22]:
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=10.0)

In [23]:
await latiss.rem.atcamera.cmd_startImage.start()

logMessage DDS read queue is filling: 10 of 100 elements


In [24]:
await asyncio.sleep(2.0)

In [26]:
await latiss.rem.atcamera.cmd_endImage.start()
# Why is this command failing???

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=717841637, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='Ack : NO Command rejected: Invalid calibration state State{CalibrationState = ENABLED}')

In [27]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [44]:
# Take a bias to make sure everything is working
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=2123245952, ack=<SalRetCode.CMD_FAILED: -302>, error=0, result='Error : MCM command failed: CCSTakeImagesCommand{numImages=1, expTime=0.0, shutter=false, sensors=, obsNote=, keyValueMap=groupId: 2021-07-02T18:45:31.694,imageType: BIAS}')

Step 2:
The following commands should take a calibration image\
Try this first without the discardRows()\
Trying again to see if it enters CalibrationIntegrating state and if it stays there.

In [31]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [32]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [33]:
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST" 

In [34]:
print(keyValueMap)

groupId: 2021-07-02T18:40:47.608,imageType: ENGTEST


In [35]:
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=10.0)

In [36]:
await latiss.rem.atcamera.cmd_startImage.start()

logMessage DDS read queue is filling: 16 of 100 elements


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1919277888, ack=<SalRetCode.CMD_FAILED: -302>, error=0, result='Error : MCM command failed: CCSStartImageCommand{shutter=true, timeout=10.0, sensors=, keyValueMap=groupId: 2021-07-02T18:40:47.608,imageType: ENGTEST, annotation=}')

In [24]:
await asyncio.sleep(2.0)

In [39]:
await latiss.rem.atcamera.cmd_endImage.start()
# Why is this command failing???

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=717841638, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='Ack : NO Command rejected: Invalid calibration state State{CalibrationState = DISABLED}')

In [37]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [56]:
# Take a bias to make sure everything is working
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021070200036])

Step 3:
The following commands should take a calibration image\
With the discardRows().\
We expect this might fail.

In [ ]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [ ]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [ ]:
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST" 

In [ ]:
print(keyValueMap)

In [ ]:
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=10.0)

In [ ]:
await latiss.rem.atcamera.cmd_startImage.start()

In [ ]:
await asyncio.sleep(2.0)

In [ ]:
latiss.rem.atcamera.cmd_discardRows.set(nRows=500)

In [ ]:
await latiss.rem.atcamera.cmd_discardRows.start()

In [ ]:
await asyncio.sleep(1.0)

In [ ]:
await latiss.rem.atcamera.cmd_endImage.start()

In [ ]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [ ]:
# Take a bias to make sure everything is working
await latiss.take_bias(1)

In [57]:
# Putting everything back in standby.
await latiss.standby()

[atcamera]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[atspectrograph]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[atheaderservice]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[atarchiver]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


Stuff below is just in case it is needed.

In [ ]:
await latiss.rem.atcamera.cmd_disable.set_start()

In [ ]:
latiss.rem.atcamera.cmd_endImage.set_start()

In [ ]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [52]:
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.ENABLED)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [47]:
test = await latiss.rem.atcamera.evt_heartbeat.next(flush=True, timeout=5)

In [46]:
print(test)

private_revCode: 0863a6ef, private_sndStamp: 1625251960.2224035, private_rcvStamp: 1625251960.2227683, private_seqNum: 9689, private_identity: ATCamera, private_origin: 98186947, private_host: 0, heartbeat: True, priority: 1


In [48]:
print(test)

private_revCode: 0863a6ef, private_sndStamp: 1625251964.2224042, private_rcvStamp: 1625251964.2227507, private_seqNum: 9693, private_identity: ATCamera, private_origin: 98186947, private_host: 0, heartbeat: True, priority: 1


In [49]:
await latiss.rem.atcamera.cmd_disable?

Type:        RemoteCommand
String form: RemoteCommand(ATCamera, 0, disable)
File:        /opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.6.0/lib/python3.8/site-packages/lsst/ts/salobj/topics/remote_command.py
Docstring:  
Issue a specific command topic and wait for acknowldgement.

Parameters
----------
salinfo : `.SalInfo`
    SAL component information
name : `str`
    Command name


In [ ]:
await latiss.rem.atcamera.cmd_disable

In [50]:
await latiss.rem.atcamera.cmd_disable.set_start()

In [ ]:
await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.STANDBY)

In [ ]:
await latiss.rem.atspectrograph.cmd_enable.set_start({"atspectrograph": "current"})

In [ ]:
await latiss.rem.atspectrograph.enable({"atspectrograph": "current"})

In [ ]:
await latiss.enable({"atspectrograph": "current"})

In [ ]:
await latiss.rem.atcamera.cmd_initImage.set_start(deltaT=2.0)

In [ ]:
await latiss.rem.atcamera.cmd_abort.start()

In [ ]:
await latiss.rem.atcamera.cmd_stop.start()

In [ ]:
await latiss.rem.atcamera.cmd_stop

In [30]:
await latiss.rem.atcamera.cmd_endImage.set?

Signature: latiss.rem.atcamera.cmd_endImage.set(**kwargs)
Docstring:
Create a new ``self.data`` and set one or more fields.

Parameters
----------
**kwargs : `dict` [`str`, ``any``]
    Dict of field name: new value for that field:

    * Any key whose value is `None` is checked for existence,
      but the value of the field is not changed.
    * If the field being set is an array then the value must either
      be an array of the same length or a scalar (which replaces
      every element of the array).

Returns
-------
did_change : `bool`
    True if ``self.data`` was changed, or if this was the first call
    to `set`.

Raises
------
AttributeError
    If the topic does not have the specified field.
ValueError
    If the field cannot be set to the specified value.

Notes
-----
If one or more fields cannot be set, the data may be partially updated.
File:      /opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.6.0/lib/python3.8/site-packages/lsst/ts/salobj/topi

In [ ]:
await latiss.rem.atcamera.cmd_endImage.set

In [ ]:
await latiss.rem.atcamera.cmd_endImage